In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import joblib


In [6]:

# Load data và model đã train sẵn
reviews = pd.read_excel("Du lieu cung cap/Reviews.xlsx")
overview_companies = pd.read_excel("Du lieu cung cap/Overview_Companies.xlsx")
overview_reviews = pd.read_excel("Du lieu cung cap/Overview_Reviews.xlsx")


In [8]:

sentiment_model = joblib.load("sentiment_model.pkl") # Model phân tích cảm xúc
cluster_model = joblib.load("cluster_model.pkl")     # Model phân cụm
cluster_keywords = joblib.load("cluster_keywords.pkl") # Dict cụm: top keywords

In [9]:

st.sidebar.title("Phân tích công ty trên ITViec")
company_names = overview_companies["Company Name"].unique()
selected_company = st.sidebar.selectbox("Chọn công ty", company_names)
tab = st.sidebar.radio("Chọn phân tích", ("Phân tích cảm xúc", "Phân cụm đánh giá"))

# Lọc dữ liệu công ty
company_reviews = reviews[reviews["Company Name"] == selected_company]

2025-06-27 23:24:30.414 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-27 23:24:30.593 
  command:

    streamlit run C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-06-27 23:24:30.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-27 23:24:30.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-27 23:24:30.596 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-27 23:24:30.596 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-27 23:24:30.597 Thread 'MainThread': missing ScriptRunContext! This w

In [12]:
if tab == "Phân tích cảm xúc":
    st.header(f"Phân tích cảm xúc review cho {selected_company}")

    # Kiểm tra xem có review nào không
    if len(company_reviews) == 0 or company_reviews['What I liked'].notna().sum() == 0:
        st.warning(f"Không có review nào cho công ty {selected_company} hoặc tất cả review đều trống.")
    else:
        # Dự đoán cảm xúc các review
        filtered_reviews = company_reviews[company_reviews['What I liked'].notna()]
        filtered_reviews['Sentiment'] = sentiment_model.predict(filtered_reviews['What I liked'])
        
        # Gán kết quả dự đoán vào company_reviews ban đầu
        company_reviews = company_reviews.copy()
        company_reviews.loc[company_reviews['What I liked'].notna(), 'Sentiment'] = filtered_reviews['Sentiment'].values

        # Thống kê số lượng từng nhãn
        st.subheader("Tổng quan cảm xúc")
        st.bar_chart(company_reviews['Sentiment'].value_counts())

        # Wordcloud cho từng nhóm cảm xúc
        for sentiment in ['positive', 'negative', 'neutral']:
            text = " ".join(company_reviews[company_reviews['Sentiment']==sentiment]['What I liked'].fillna(''))
            if text.strip():  # Kiểm tra text không rỗng
                wc = WordCloud(width=800, height=400).generate(text)
                st.subheader(f"Wordcloud cho review {sentiment}")
                st.image(wc.to_array())
        
        # Bảng các review và cảm xúc
        st.subheader("Các review và cảm xúc")
        st.dataframe(company_reviews[['Title', 'What I liked', 'Sentiment']])
        
elif tab == "Phân cụm đánh giá":
    st.header(f"Phân cụm đánh giá cho {selected_company}")

    # Lấy index cụm công ty này thuộc về
    company_idx = overview_companies[overview_companies['Company Name']==selected_company].index[0]
    cluster_label = cluster_model.labels_[company_idx]
    st.subheader(f"Công ty này thuộc cụm: {cluster_label}")

    # Đặc trưng, top keywords của cụm
    st.markdown("**Đặc trưng cụm:**")
    st.write(cluster_keywords[cluster_label])

    # Pie chart tỷ lệ cụm trên toàn bộ dataset
    cluster_counts = pd.Series(cluster_model.labels_).value_counts()
    st.subheader("Tỷ lệ các cụm trên toàn bộ dữ liệu")
    st.pyplot(plt.figure().gca().pie(cluster_counts, labels=cluster_counts.index, autopct='%1.1f%%'))

    # Đề xuất cải thiện cho cụm này (bổ sung thêm)
    st.markdown("**Đề xuất cải thiện:**")
    # (Thêm phần này theo gợi ý của bạn, ví dụ từ bảng mapping cluster <-> improvement)

    # Show review của công ty
    st.subheader("Các review nổi bật của công ty")
    st.dataframe(company_reviews[['Title', 'What I liked']])



2025-06-27 23:26:38.326 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-27 23:26:38.328 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-27 23:26:38.328 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-27 23:26:38.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-27 23:26:38.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-27 23:26:38.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
